In [1]:
import mne
import os
from mne.time_frequency import psd_array_multitaper
import numpy as np
from scipy.stats import entropy, skew, kurtosis
import csv



In [2]:
import mne
import os
from mne.time_frequency import psd_array_multitaper
import numpy as np
from scipy.stats import entropy, skew, kurtosis

def read_eeg_files(directory):
    eeg_data = []

    for filename in os.listdir(directory):
        if filename.endswith('.edf'):
            filepath = os.path.join(directory, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            eeg_data.append(raw)

    return eeg_data

def extract_spectral_features(raw, freq_bands):
    spectral_features = []

    raw.filter(1, 120)
    raw.set_eeg_reference('average')
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    psds, freqs = psd_array_multitaper(raw.get_data()[eeg_picks], sfreq=raw.info['sfreq'], fmin=0.5, fmax=30) # type: ignore

    for band, freq_range in freq_bands.items():
        band_data = raw.copy().filter(freq_range[0], freq_range[1], picks=eeg_picks).get_data()

        if np.isnan(band_data).any():
            print(f"No data available for {band} band.")
            continue

        band_power = np.mean(np.abs(band_data) ** 2, axis=1)
        band_entropy = entropy(np.abs(band_data) ** 2, axis=1)
        band_skewness = skew(band_data, axis=1)

        spectral_features += [band_power, band_entropy, band_skewness]


    return spectral_features

def extract_time_features(raw):
    time_features = []

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    eeg_data = raw.get_data()[eeg_picks]

    for channel_data in eeg_data:
        channel_mean = np.mean(channel_data)

        channel_std = np.std(channel_data)

        channel_kurtosis = kurtosis(channel_data)
            
        channel_ptp = np.ptp(channel_data,axis=-1)

        channel_var = np.var(channel_data,axis=-1)

        channel_min = np.min(channel_data,axis=-1)

        channel_max = np.max(channel_data,axis=-1)

        channel_argmin = np.argmin(channel_data,axis=-1)

        channel_argmax = np.argmax(channel_data,axis=-1)

        channel_rms= np.sqrt(np.mean(channel_data ** 2,axis=-1))  #root mean sqaured #type: ignore


        time_features +=[channel_mean,channel_std,channel_kurtosis,channel_ptp,channel_var,channel_min,channel_max,channel_argmin,channel_argmax,channel_rms]

    return time_features



In [3]:

# Example usage:


def save_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

# Example usage:
directory = 'Modeltraning\CH08'

eeg_data = read_eeg_files(directory)
freq_bands = {'delta': [0.5, 4], 'theta': [4, 7], 'alpha': [7, 15], 'beta': [15, 30]}

for raw in eeg_data:
    spectral_features = extract_spectral_features(raw, freq_bands)
    time_features = extract_time_features(raw)

    # Combine spectral and time features into a single list
    all_features = spectral_features + time_features

    # Save features to a CSV file
    save_to_csv(all_features, 'features.csv')


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Modeltraning\\CH08'